# Accessing Bacdive API
There is an installable tool [bacdive](https://github.com/JKoblitz/bacdive-api/blob/master/bacdive/client.py) that allows for queries of identifiers in bacdive by:
- taxonomy
- 16s
- full genome
- culture number
And to get all information from bacdive ID's returned from the above. However, I observed on the web API that one can search for NCBI taxid. When doing so, the response is a redirect, but the header contains the bacdive ID. It would be nice to add this functionality.

## Let's get the OGT based on taxonomy extracted from a refseq file using the base API

In [1]:
from Bio import SeqIO
import bacdive
import os

In [3]:
genome = SeqIO.parse('../data/refseq/archaea/GCF_021655615.1_ASM2165561v1_genomic.gbff', 'genbank')
record = next(genome)

In [4]:
client = bacdive.BacdiveClient(os.environ['ENV_EMAIL'], os.environ['BACDIVE_PASSWORD'])

-- Authentication successful --


In [5]:
client.result = client.getIDsByTaxonomy(record.annotations['taxonomy'][5])

In [6]:
response = client.retrieve()

In [7]:
responses = [r for r in response]

In [8]:
for r in responses:
    string = r['Name and taxonomic classification']['species']
    if 'strain designation' in  r['Name and taxonomic classification']:
        string += f" {r['Name and taxonomic classification']['strain designation']}"
    string += f" {str(r['General']['NCBI tax id'])}"
    print(string)

Acidianus sp. RZ1 {'NCBI tax id': 1540082, 'Matching level': 'species'}
Acidianus sp. {'NCBI tax id': 1872104, 'Matching level': 'species'}
Acidianus sulfidivorans JP7 [{'NCBI tax id': 619593, 'Matching level': 'strain'}, {'NCBI tax id': 312539, 'Matching level': 'species'}]
Acidianus ambivalens {'NCBI tax id': 2283, 'Matching level': 'species'}
Acidianus infernus So4a {'NCBI tax id': 12915, 'Matching level': 'species'}
Acidianus brierleyi Sp3a/1 {'NCBI tax id': 41673, 'Matching level': 'species'}
Acidianus brierleyi {'NCBI tax id': 41673, 'Matching level': 'species'}


Using "taxonomy" search, on this refseq example, we actually get back multiple results, each with and NCBI Id

In [9]:
record.features[0].qualifiers['db_xref']

['taxon:2886040']

None of the returned TAXID match the id on the file from refseq.
> As a result __Bacdive Taxonomy search is too loose__

*However* the Taxid for species sp. and strain RZ1 is 1540082, __which does exist in refseq__

> Thus, we should search Bacdive by NCBI tax ID, and not all refseq examples with have data. It remaims to be seen if this method will produce a sufficient number of OGTs.

## Search by TAXID

We can extract TAXID from refseq files in the "source" feature of the record in the dictionary qualifiers. Let's try to use such an ID to get a bacdive record.

In [10]:
class BacdiveClient(bacdive.BacdiveClient):

    def getIDByNCBITaxID(self, tid):
        if self.public:
            baseurl = "https://bacdive.dsmz.de/"
        else:
            baseurl = "http://bacdive-dev.dsmz.local/"
        url = baseurl+"search?search=taxid:"+str(tid)
        print(url)
        resp = self.do_request(url)

        # now get the strain ID from the response
        sid = int(resp.history[0].headers['location'].split('/')[-1])

        result = {'count': 1,
            'next': None,
            'previous': None,
            'results': [sid]}
        self.result = result
        return 1

In [11]:
tid = 1540082

In [12]:
client = BacdiveClient(os.environ['ENV_EMAIL'], os.environ['BACDIVE_PASSWORD'])

-- Authentication successful --


In [13]:
client.getIDByNCBITaxID(tid)

https://bacdive.dsmz.de/search?search=taxid:1540082


1

In [14]:
results = [i for i in client.retrieve()]

In [15]:
results

[{'General': {'@ref': 22551,
   'BacDive-ID': 131324,
   'DSM-Number': 29099,
   'keywords': ['thermophilic', 'Archaea', '16S sequence', 'genome sequence'],
   'description': 'Acidianus sp. RZ1 is a thermophilic archaeon that was isolated from hot spring.',
   'NCBI tax id': {'NCBI tax id': 1540082, 'Matching level': 'species'},
   'strain history': ['<- R. Zhang, Univ. Duisburg-Essen, Germany; RZ1 <- R. Zhang {2011}',
    'W. Sand; Aquatic Biotechnol., Biofilm Centre, Univ. of Duisburg-Essen, Germany; RZ1.'],
   'doi': '10.13145/bacdive131324.20220920.7'},
  'Name and taxonomic classification': {'LPSN': {'@ref': 20215,
    'description': 'domain/archaea',
    'keyword': 'phylum/thermoproteota',
    'domain': 'Archaea',
    'phylum': 'Thermoproteota',
    'class': 'Thermoprotei',
    'order': 'Sulfolobales',
    'family': 'Sulfolobaceae',
    'genus': 'Acidianus',
    'species': 'Acidianus sp.',
    'full scientific name': '<I>Acidianus</I> Segerer et al. 1986',
    'synonyms': {'@ref'

And there it is! A Bacdive result with the correct TAXID.

In [16]:
results[0]['Culture and growth conditions']['culture temp']

[{'@ref': 22551,
  'growth': 'positive',
  'type': 'growth',
  'temperature': '65',
  'range': 'thermophilic'},
 {'@ref': 67770,
  'growth': 'positive',
  'type': 'growth',
  'temperature': '65',
  'range': 'thermophilic'}]

### BOOM

> search bacdive by TAXID with this new modified class